In [8]:
import requests
import json

# --- CONFIGURACIÓN (Edita esto con tus datos) ---
BASE_URL = "http://localhost:8080"    # Tu URL base de Evolution API
INSTANCE_NAME = "Alejandro" # El nombre que le diste a tu instancia (ej. "Ventas")
API_KEY = "322312332231123123"         # Tu API Key (la que definiste en el .env o config)
#PHONE = "51974979081"                 # Número destino con código país (ej. 51 Perú), SIN el símbolo +
PHONE ="51974979081"

In [11]:
def enviar_mensaje(numero, mensaje):
    # ⚠️ CAMBIO 1: La URL en v2 es diferente (usa slashes /send/text/)
    url = f"{BASE_URL}/message/sendText/{INSTANCE_NAME}"

    headers = {
        "apikey": API_KEY,
        "Content-Type": "application/json"
    }

    # ⚠️ CAMBIO 2: El JSON es "plano".
    # Ya no se usa "textMessage": { "text": ... }, ahora es directo "text": ...
    payload = {
        "number": numero,
        "text": mensaje,
        "delay": 1200,        # Opcional: tiempo de escritura en ms
        "linkPreview": True   # Opcional: previsualizar enlaces
    }

    try:
        print(f"📡 Enviando a: {url}")
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 201 or response.status_code == 200:
            print("✅ Mensaje enviado con éxito.")
            print("Respuesta:", response.json())
        else:
            print(f"❌ Error. Código: {response.status_code}")
            print("Detalle:", response.text)

    except Exception as e:
        print(f"⚠️ Error de conexión: {e}")

if __name__ == "__main__":

#### MENSAJE DE TEXTO    
    mi_mensaje = "1321676898"
    enviar_mensaje(PHONE, mi_mensaje)


📡 Enviando a: http://localhost:8080/message/sendText/Alejandro
✅ Mensaje enviado con éxito.
Respuesta: {'key': {'remoteJid': '51974979081@s.whatsapp.net', 'fromMe': True, 'id': '3EB0F980CF72AFEF6EE4746E3443DAEA5537B142'}, 'pushName': '', 'status': 'PENDING', 'message': {'conversation': '1321676898'}, 'contextInfo': None, 'messageType': 'conversation', 'messageTimestamp': 1763999641, 'instanceId': 'b87cd66c-3ee5-4174-938d-bc6a216ec01b', 'source': 'unknown'}


In [1]:
import requests
import json
import base64
import mimetypes
import os

# --- CONFIGURACIÓN (Edita esto con tus datos) ---
BASE_URL = "http://localhost:8080"    # Tu URL base de Evolution API
INSTANCE_NAME = "Alejandro" # El nombre que le diste a tu instancia (ej. "Ventas")
API_KEY = "322312332231123123"         # Tu API Key (la que definiste en el .env o config)
#PHONE = "51974979081"                 # Número destino con código país (ej. 51 Perú), SIN el símbolo +
PHONE ="51974979081"

In [3]:

def enviar_archivo(numero, ruta_archivo, mensaje_caption):
    
    # 1. Verificar existencia
    if not os.path.exists(ruta_archivo):
        print(f"❌ Error: No encuentro el archivo en: {ruta_archivo}")
        return

    # 2. Detectar tipo MIME y preparar Base64
    mime_type, _ = mimetypes.guess_type(ruta_archivo)
    if mime_type is None:
        mime_type = "application/octet-stream"

    nombre_archivo = os.path.basename(ruta_archivo)

    try:
        with open(ruta_archivo, "rb") as file:
            # Leemos y codificamos
            raw_b64 = base64.b64encode(file.read()).decode('utf-8')
            
            # ⚠️ FIX IMPORTANTE: Eliminamos saltos de línea que rompen el JSON
            raw_b64 = raw_b64.replace("\n", "").replace("\r", "")
            
            # Creamos el Data URI completo
            media_data = f"data:{mime_type};base64,{raw_b64}"
            
    except Exception as e:
        print(f"❌ Error leyendo archivo: {e}")
        return

    # 3. Endpoint Nativo V2
    # Nota: En v2 la URL usa /send/media/ (con slashes)
    url = f"{BASE_URL}/message/sendMedia/{INSTANCE_NAME}"

    headers = {
        "apikey": API_KEY,
        "Content-Type": "application/json"
    }

    # Determinar tipo de medio para la API
    tipo_medio = "document" # Por defecto
    if mime_type.startswith("image/"):
        tipo_medio = "image"
    elif mime_type.startswith("video/"):
        tipo_medio = "video"

    # 4. Payload V2 estricto
    payload = {
        "number": numero,
        "media": media_data,       # La cadena base64 limpia
        "mediatype": tipo_medio,   # "image", "document", etc.
        "mimetype": mime_type,     # Añadimos esto explícitamente (ayuda a la validación)
        "caption": mensaje_caption,
        "fileName": nombre_archivo
    }

    try:
        print(f"📡 Enviando {nombre_archivo} ({len(media_data)} bytes)...")
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code in [200, 201]:
            print("✅ Archivo enviado con éxito.")
            print(response.json())
        else:
            print(f"❌ Error ({response.status_code}):")
            print(response.text)

    except Exception as e:
        print(f"⚠️ Error de conexión: {e}")

if __name__ == "__main__":
    # Asegúrate de que la ruta sea correcta y el archivo NO ESTÉ VACÍO.
    # Si usas Windows, pon una 'r' antes de las comillas.
    ruta_pdf = r"C:/Users/IRVIN/Downloads/123.pdf" 
    
    enviar_archivo(PHONE, ruta_pdf, "Prueba de PDF corregida 📄")

📡 Enviando 123.pdf (945868 bytes)...
❌ Error (400):
{"status":400,"error":"Bad Request","response":{"message":["Owned media must be a url or base64"]}}
